In [1]:
!gcloud services enable pubsub



In [2]:
!gcloud services enable dataflow.googleapis.com


In [3]:
!pip install apache-beam[gcp]


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 355.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 182.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 483.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 402.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 179.0 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=23540 sha256=e34a0a977b930fbc7d6dc28834f6ea559c361c4553219889b5a3ad1f7bd1fae9
  Stored in directory: /tmp/pip-ephem-wheel-cache-rojwjq_b/wheels/85/4c/07/72215c529bd59d67e3dac29711d7aba1b692f543c808ba9e86
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78542 sha256=aaf8981208d4fa0ed8bebe16f967703

In [4]:
import apache_beam as beam
from apache_beam.runners import DataflowRunner
from apache_beam.transforms import trigger
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.io import WriteToBigQuery
from apache_beam.io import BigQueryDisposition
import google.auth
import json
import pandas as pd

In [5]:
from apache_beam.options.pipeline_options import PipelineOptions


In [6]:
pipeline_options = PipelineOptions(
    runner='DataflowRunner',  # Use 'DirectRunner' for local execution
    project='customer-churn-451414',
    region='asia-southeast1',
    temp_location='gs://customer-churn-storage-bucket/temp',
    staging_location='gs://customer-churn-storage-bucket/staging',
    job_name='dataflow-etl-job'
)


In [7]:
!gcloud storage buckets list



---
creation_time: 2025-02-19T14:03:29+0000
default_storage_class: STANDARD
generation: 1739973807725754051
location: ASIA
location_type: multi-region
metageneration: 2
name: customer-churn-storage-bucket
public_access_prevention: inherited
rpo: DEFAULT
soft_delete_policy:
  retentionDurationSeconds: '0'
storage_url: gs://customer-churn-storage-bucket/
uniform_bucket_level_access: true
update_time: 2025-02-25T14:19:41+0000


In [8]:
!gcloud storage buckets describe gs://customer-churn-storage-bucket


creation_time: 2025-02-19T14:03:29+0000
default_storage_class: STANDARD
generation: 1739973807725754051
location: ASIA
location_type: multi-region
metageneration: 2
name: customer-churn-storage-bucket
public_access_prevention: inherited
rpo: DEFAULT
soft_delete_policy:
  retentionDurationSeconds: '0'
storage_url: gs://customer-churn-storage-bucket/
uniform_bucket_level_access: true
update_time: 2025-02-25T14:19:41+0000


In [10]:
import apache_beam as beam

def parse_csv(line):
    import csv
    from io import StringIO
    reader = csv.reader(StringIO(line))
    return next(reader)  # Returns a list of values

with beam.Pipeline(options=pipeline_options) as p:
    raw_data = (
        p
        | 'ReadFromGCS' >> beam.io.ReadFromText('gs://customer-churn-storage-bucket/Telco_Customer_Churn.csv', skip_header_lines=1)
        | 'ParseCSV' >> beam.Map(parse_csv)
    )


ERROR:apache_beam.runners.dataflow.dataflow_runner:2025-02-27T13:27:02.322Z: JOB_MESSAGE_ERROR: Startup of the worker pool in zone asia-southeast1-a failed to bring up any of the desired 1 workers. Please refer to https://cloud.google.com/dataflow/docs/guides/common-errors#worker-pool-failure for help troubleshooting. ZONE_RESOURCE_POOL_EXHAUSTED: Instance 'dataflow-etl-job-02270526-1l4n-harness-7j1q' creation failed: The zone 'projects/customer-churn-451414/zones/asia-southeast1-a' does not have enough resources available to fulfill the request.  Try a different zone, or try again later.
ERROR:apache_beam.runners.dataflow.dataflow_runner:2025-02-27T13:27:02.339Z: JOB_MESSAGE_ERROR: Workflow failed.
ERROR:apache_beam.runners.dataflow.dataflow_runner:Console URL: https://console.cloud.google.com/dataflow/jobs/<RegionId>/2025-02-27_05_26_05-5644318696860845791?project=<ProjectId>


DataflowRuntimeException: Dataflow pipeline failed. State: FAILED, Error:
Workflow failed.

In [ ]:
import apache_beam as beam
import pandas as pd
from apache_beam.options.pipeline_options import PipelineOptions

pipeline_options = PipelineOptions([
    '--runner=DirectRunner'  # Local execution
])

def parse_csv(line):
    import csv
    from io import StringIO
    reader = csv.reader(StringIO(line))
    return next(reader)  # Returns a list of values

column_names = [
    "customerID", "gender", "SeniorCitizen", "Partner", "Dependents",
    "tenure", "PhoneService", "MultipleLines", "InternetService",
    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling",
    "PaymentMethod", "MonthlyCharges", "TotalCharges", "Churn"
]

output_prefix = "transformed_output.txt"

with beam.Pipeline(options=pipeline_options) as p:
    data = (
        p
        | 'ReadFromGCS' >> beam.io.ReadFromText(
            'gs://customer-churn-storage-bucket/Telco_Customer_Churn.csv', 
            skip_header_lines=1
        )
        | 'ParseCSV' >> beam.Map(parse_csv)
    )

    # Add header row
    header_pcoll = p | 'CreateHeader' >> beam.Create([column_names])
    full_data = (header_pcoll, data) | 'CombineHeaderWithData' >> beam.Flatten()

    # Write to text
    full_data | 'WriteToText' >> beam.io.WriteToText(output_prefix, shard_name_template="")

print("Pipeline execution completed.")

In [11]:
df = pd.read_csv(f"{output_file}-00000-of-00001", header=None)
print("Column Names:", df.columns.tolist())


NameError: name 'output_file' is not defined

In [12]:
df = pd.read_csv(f"{output_file}-00000-of-00001", header=None)
print(df.head())  # Show first few rows

NameError: name 'output_file' is not defined

In [13]:
import os
print(os.listdir())  # List all files in the directory


['.ipython', '00-Start_Here.md', 'Tutorials', '.local', '.jupyter', 'transformed_output.txt', '.ipynb_checkpoints', 'Examples', 'assets', '.config', 'faq.md', 'output.txt-00000-of-00001', 'Customer-churn.ipynb', '.cache']


In [14]:
import pandas as pd

# Define column names manually
column_names = [
    "customerID", "gender", "SeniorCitizen", "Partner", "Dependents",
    "tenure", "PhoneService", "MultipleLines", "InternetService",
    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling",
    "PaymentMethod", "MonthlyCharges", "TotalCharges", "Churn"
]

# Read the Beam output file
output_file = "output.txt-00000-of-00001"  # Make sure this file exists!
df = pd.read_csv(output_file, header=None, names=column_names)

# Print column names
print("Column Names:", df.columns.tolist())

# Print first few rows
print(df.head())


Column Names: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
      customerID     gender SeniorCitizen Partner Dependents tenure  \
0  ['7590-VHVEG'   'Female'           '0'   'Yes'       'No'    '1'   
1  ['5575-GNVDE'     'Male'           '0'    'No'       'No'   '34'   
2  ['3668-QPYBK'     'Male'           '0'    'No'       'No'    '2'   
3  ['7795-CFOCW'     'Male'           '0'    'No'       'No'   '45'   
4  ['9237-HQITU'   'Female'           '0'    'No'       'No'    '2'   

  PhoneService        MultipleLines InternetService OnlineSecurity  ...  \
0         'No'   'No phone service'           'DSL'           'No'  ...   
1        'Yes'                 'No'           'DSL'          'Yes'  ...   
2        '

In [15]:
import apache_beam as beam
import pandas as pd
from apache_beam.options.pipeline_options import PipelineOptions

# Define pipeline options
pipeline_options = PipelineOptions([
    '--runner=DirectRunner'  # Use DirectRunner for local execution
])

# Define column names
column_names = [
    "customerID", "gender", "SeniorCitizen", "Partner", "Dependents",
    "tenure", "PhoneService", "MultipleLines", "InternetService",
    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling",
    "PaymentMethod", "MonthlyCharges", "TotalCharges", "Churn"
]

# Function to parse CSV row
def parse_csv(line):
    import csv
    from io import StringIO
    reader = csv.reader(StringIO(line))
    return next(reader)  # Returns a list of values

# Function to clean & transform data
def transform_data(row):
    try:
        # Convert to dictionary
        data = dict(zip(column_names, row))

        # Convert 'TotalCharges' to numeric (handle missing values)
        try:
            data["TotalCharges"] = float(data["TotalCharges"]) if data["TotalCharges"].strip() else 0.0
        except ValueError:
            data["TotalCharges"] = 0.0

        # Convert binary categorical columns (Yes/No → 1/0)
        binary_cols = ["Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]
        for col in binary_cols:
            data[col] = 1 if data[col].strip().lower() == "yes" else 0

        # One-Hot Encode multi-class categorical variables
        one_hot_cols = ["InternetService", "Contract", "PaymentMethod"]
        for col in one_hot_cols:
            value = data[col].replace(" ", "_").lower()  # Normalize column names
            data[f"{col}_{value}"] = 1  # One-hot encode
            del data[col]  # Remove original column

        # Convert 'SeniorCitizen' to categorical
        data["SeniorCitizen"] = int(data["SeniorCitizen"])

        # Feature Engineering: AvgMonthlySpend = TotalCharges / tenure
        tenure = int(data["tenure"])
        data["AvgMonthlySpend"] = round(data["TotalCharges"] / tenure, 2) if tenure > 0 else 0.0

        # Create tenure groups
        if tenure < 12:
            data["TenureGroup"] = "New"
        elif 12 <= tenure < 48:
            data["TenureGroup"] = "Medium"
        else:
            data["TenureGroup"] = "LongTerm"

        # Scale continuous variables (Min-Max Scaling)
        data["MonthlyCharges"] = round(float(data["MonthlyCharges"]) / 100, 2)
        data["TotalCharges"] = round(float(data["TotalCharges"]) / 10000, 4)

        return data  # Return transformed data as dictionary

    except Exception as e:
        print(f"Error processing row: {row} - {e}")
        return None

# Define Apache Beam pipeline
with beam.Pipeline(options=pipeline_options) as p:
    transformed_data = (
        p
        | 'ReadFromGCS' >> beam.io.ReadFromText(
            'gs://customer-churn-storage-bucket/Telco_Customer_Churn.csv', 
            skip_header_lines=1
        )
        | 'ParseCSV' >> beam.Map(parse_csv)
        | 'TransformData' >> beam.Map(transform_data)
        | 'RemoveNone' >> beam.Filter(lambda x: x is not None)  # Remove failed transformations
        | 'WriteToText' >> beam.io.WriteToText("transformed_output.txt", shard_name_template="")
    )

print("Data transformation complete.")


Data transformation complete.


In [18]:
!gcloud compute zones list | grep asia-southeast1


asia-southeast1-b          asia-southeast1          UP
asia-southeast1-a          asia-southeast1          UP
asia-southeast1-c          asia-southeast1          UP


In [28]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.io.gcp.bigquery import WriteToBigQuery, BigQueryDisposition
from google.cloud import bigquery

# Define pipeline options
pipeline_options = PipelineOptions([
    '--runner=DirectRunner',
    '--project=customer-churn-451414',
    '--temp_location=gs://customer-churn-storage-bucket/temp',
])

# Define column names
column_names = [
    "customerID", "gender", "SeniorCitizen", "Partner", "Dependents",
    "tenure", "PhoneService", "MultipleLines", "InternetService",
    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling",
    "PaymentMethod", "MonthlyCharges", "TotalCharges", "Churn"
]

# Function to parse CSV row
def parse_csv(line):
    import csv
    from io import StringIO
    reader = csv.reader(StringIO(line))
    return next(reader)

# Function to clean & transform data (updated to ensure all one-hot fields are present)
def transform_data(row):
    try:
        data = dict(zip(column_names, row))
        try:
            data["TotalCharges"] = float(data["TotalCharges"]) if data["TotalCharges"].strip() else 0.0
        except ValueError:
            data["TotalCharges"] = 0.0
        binary_cols = ["Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]
        for col in binary_cols:
            data[col] = 1 if data[col].strip().lower() == "yes" else 0
        
        # Define all possible one-hot encoded values and initialize to 0
        one_hot_values = {
            "InternetService": ["dsl", "fiber_optic", "no"],
            "Contract": ["month-to-month", "one_year", "two_year"],
            "PaymentMethod": ["electronic_check", "mailed_check", "bank_transfer_automatic", "credit_card_automatic"]
        }
        
        # Initialize all one-hot fields to 0
        for col, values in one_hot_values.items():
            for value in values:
                data[f"{col}_{value}"] = 0
        
        # Set the actual one-hot encoded value to 1
        one_hot_cols = ["InternetService", "Contract", "PaymentMethod"]
        for col in one_hot_cols:
            value = data[col].replace(" ", "_").replace("(", "_").replace(")", "_").lower()
            if f"{col}_{value}" in data:  # Ensure the key exists in the initialized set
                data[f"{col}_{value}"] = 1
            del data[col]
        
        data["SeniorCitizen"] = int(data["SeniorCitizen"])
        tenure = int(data["tenure"])
        data["AvgMonthlySpend"] = round(data["TotalCharges"] / tenure, 2) if tenure > 0 else 0.0
        if tenure < 12:
            data["TenureGroup"] = "New"
        elif 12 <= tenure < 48:
            data["TenureGroup"] = "Medium"
        else:
            data["TenureGroup"] = "LongTerm"
        data["MonthlyCharges"] = round(float(data["MonthlyCharges"]) / 100, 2)
        data["TotalCharges"] = round(float(data["TotalCharges"]) / 10000, 4)
        return data
    except Exception as e:
        print(f"Error processing row: {row} - {e}")
        return None

# Define BigQuery schema
bq_schema = {
    "fields": [
        {"name": "customerID", "type": "STRING"},
        {"name": "gender", "type": "STRING"},
        {"name": "SeniorCitizen", "type": "INTEGER"},
        {"name": "Partner", "type": "INTEGER"},
        {"name": "Dependents", "type": "INTEGER"},
        {"name": "tenure", "type": "INTEGER"},
        {"name": "PhoneService", "type": "INTEGER"},
        {"name": "MultipleLines", "type": "STRING"},
        {"name": "OnlineSecurity", "type": "STRING"},
        {"name": "OnlineBackup", "type": "STRING"},
        {"name": "DeviceProtection", "type": "STRING"},
        {"name": "TechSupport", "type": "STRING"},
        {"name": "StreamingTV", "type": "STRING"},
        {"name": "StreamingMovies", "type": "STRING"},
        {"name": "PaperlessBilling", "type": "INTEGER"},
        {"name": "MonthlyCharges", "type": "FLOAT"},
        {"name": "TotalCharges", "type": "FLOAT"},
        {"name": "Churn", "type": "INTEGER"},
        {"name": "InternetService_dsl", "type": "INTEGER"},
        {"name": "InternetService_fiber_optic", "type": "INTEGER"},
        {"name": "InternetService_no", "type": "INTEGER"},
        {"name": "Contract_month-to-month", "type": "INTEGER"},
        {"name": "Contract_one_year", "type": "INTEGER"},
        {"name": "Contract_two_year", "type": "INTEGER"},
        {"name": "PaymentMethod_electronic_check", "type": "INTEGER"},
        {"name": "PaymentMethod_mailed_check", "type": "INTEGER"},
        {"name": "PaymentMethod_bank_transfer_automatic", "type": "INTEGER"},
        {"name": "PaymentMethod_credit_card_automatic", "type": "INTEGER"},
        {"name": "AvgMonthlySpend", "type": "FLOAT"},
        {"name": "TenureGroup", "type": "STRING"}
    ]
}

# Function to create the BigQuery dataset if it doesn’t exist
def create_dataset(project_id, dataset_id, location="asia-southeast1"):
    client = bigquery.Client(project=project_id)
    dataset_ref = f"{project_id}.{dataset_id}"
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset {dataset_ref} already exists.")
    except Exception:
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = location
        client.create_dataset(dataset)
        print(f"Created dataset {dataset_ref} in location {location}.")

# Create the dataset before running the pipeline
create_dataset("customer-churn-451414", "customer_churn_data", "asia-southeast1")

# Define Apache Beam pipeline
with beam.Pipeline(options=pipeline_options) as p:
    transformed_data = (
        p
        | 'ReadFromGCS' >> beam.io.ReadFromText(
            'gs://customer-churn-storage-bucket/Telco_Customer_Churn.csv', 
            skip_header_lines=1
        )
        | 'ParseCSV' >> beam.Map(parse_csv)
        | 'TransformData' >> beam.Map(transform_data)
        | 'RemoveNone' >> beam.Filter(lambda x: x is not None)
        # Temporarily write to text for debugging
        | 'WriteToText' >> beam.io.WriteToText("transformed_output.txt", shard_name_template="")
        # Uncomment the line below after verifying the output
        #| 'WriteToBigQuery' >> WriteToBigQuery(
             #table='customer-churn-451414:customer_churn_data.customer_churn',
             #schema=bq_schema,
             #write_disposition=BigQueryDisposition.WRITE_TRUNCATE,
             #create_disposition=BigQueryDisposition.CREATE_IF_NEEDED,
             #custom_gcs_temp_location='gs://customer-churn-storage-bucket/temp'
         #)
    )

print("Data transformation complete. Check transformed_output.txt for debugging.")

Dataset customer-churn-451414.customer_churn_data already exists.


Data transformation complete. Check transformed_output.txt for debugging.


In [32]:
# Download the file locally
!gsutil cp gs://customer-churn-storage-bucket/temp/bq_load/d69da035646a4e0d96538b6ed8431e1e/customer-churn-451414.customer_churn_data.customer_churn/5768159a-05bb-478c-acd9-3ef213193b7e temp.json

# Display the first 5 lines
with open("temp.json", "r") as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip())
        else:
            break

Copying gs://customer-churn-storage-bucket/temp/bq_load/d69da035646a4e0d96538b6ed8431e1e/customer-churn-451414.customer_churn_data.customer_churn/5768159a-05bb-478c-acd9-3ef213193b7e...
/ [1 files][  3.7 MiB/  3.7 MiB]                                                
Operation completed over 1 objects/3.7 MiB.                                      
{"customerID": "7590-VHVEG", "gender": "Female", "SeniorCitizen": 0, "Partner": 1, "Dependents": 0, "tenure": "1", "PhoneService": 0, "MultipleLines": "No phone service", "OnlineSecurity": "No", "OnlineBackup": "Yes", "DeviceProtection": "No", "TechSupport": "No", "StreamingTV": "No", "StreamingMovies": "No", "PaperlessBilling": 1, "MonthlyCharges": 0.3, "TotalCharges": 0.003, "Churn": 0, "InternetService_dsl": 1, "Contract_month-to-month": 1, "PaymentMethod_electronic_check": 1, "AvgMonthlySpend": 29.85, "TenureGroup": "New"}
{"customerID": "5575-GNVDE", "gender": "Male", "SeniorCitizen": 0, "Partner": 0, "Dependents": 0, "tenure": "34", "Pho

In [38]:
!gsutil cp gs://customer-churn-storage-bucket/temp/bq_load/d69da035646a4e0d96538b6ed8431e1e/customer-churn-451414.customer_churn_data.customer_churn/5768159a-05bb-478c-acd9-3ef213193b7e temp.json
with open("temp.json", "r") as f:
    for i, line in enumerate(f):
        if i < 5:
            print(line.strip())
        else:
            break

Copying gs://customer-churn-storage-bucket/temp/bq_load/d69da035646a4e0d96538b6ed8431e1e/customer-churn-451414.customer_churn_data.customer_churn/5768159a-05bb-478c-acd9-3ef213193b7e...
/ [1 files][  3.7 MiB/  3.7 MiB]                                                
Operation completed over 1 objects/3.7 MiB.                                      
{"customerID": "7590-VHVEG", "gender": "Female", "SeniorCitizen": 0, "Partner": 1, "Dependents": 0, "tenure": "1", "PhoneService": 0, "MultipleLines": "No phone service", "OnlineSecurity": "No", "OnlineBackup": "Yes", "DeviceProtection": "No", "TechSupport": "No", "StreamingTV": "No", "StreamingMovies": "No", "PaperlessBilling": 1, "MonthlyCharges": 0.3, "TotalCharges": 0.003, "Churn": 0, "InternetService_dsl": 1, "Contract_month-to-month": 1, "PaymentMethod_electronic_check": 1, "AvgMonthlySpend": 29.85, "TenureGroup": "New"}
{"customerID": "5575-GNVDE", "gender": "Male", "SeniorCitizen": 0, "Partner": 0, "Dependents": 0, "tenure": "34", "Pho

In [39]:
import json

bq_schema = [
    {"name": "customerID", "type": "STRING"},
    {"name": "gender", "type": "STRING"},
    {"name": "SeniorCitizen", "type": "INTEGER"},
    {"name": "Partner", "type": "INTEGER"},
    {"name": "Dependents", "type": "INTEGER"},
    {"name": "tenure", "type": "INTEGER"},
    {"name": "PhoneService", "type": "INTEGER"},
    {"name": "MultipleLines", "type": "STRING"},
    {"name": "OnlineSecurity", "type": "STRING"},
    {"name": "OnlineBackup", "type": "STRING"},
    {"name": "DeviceProtection", "type": "STRING"},
    {"name": "TechSupport", "type": "STRING"},
    {"name": "StreamingTV", "type": "STRING"},
    {"name": "StreamingMovies", "type": "STRING"},
    {"name": "PaperlessBilling", "type": "INTEGER"},
    {"name": "MonthlyCharges", "type": "FLOAT"},
    {"name": "TotalCharges", "type": "FLOAT"},
    {"name": "Churn", "type": "INTEGER"},
    {"name": "InternetService_dsl", "type": "INTEGER"},
    {"name": "InternetService_fiber_optic", "type": "INTEGER"},
    {"name": "InternetService_no", "type": "INTEGER"},
    {"name": "Contract_month-to-month", "type": "INTEGER"},
    {"name": "Contract_one_year", "type": "INTEGER"},
    {"name": "Contract_two_year", "type": "INTEGER"},
    {"name": "PaymentMethod_electronic_check", "type": "INTEGER"},
    {"name": "PaymentMethod_mailed_check", "type": "INTEGER"},
    {"name": "PaymentMethod_bank_transfer__automatic_", "type": "INTEGER"},  # Double underscore
    {"name": "PaymentMethod_credit_card__automatic_", "type": "INTEGER"},    # Double underscore
    {"name": "AvgMonthlySpend", "type": "FLOAT"},
    {"name": "TenureGroup", "type": "STRING"}
]

with open("bq_schema.json", "w") as f:
    json.dump(bq_schema, f)

In [40]:
!bq load \
  --source_format=NEWLINE_DELIMITED_JSON \
  customer-churn-451414:customer_churn_data.customer_churn \
  gs://customer-churn-storage-bucket/temp/bq_load/d69da035646a4e0d96538b6ed8431e1e/customer-churn-451414.customer_churn_data.customer_churn/5768159a-05bb-478c-acd9-3ef213193b7e \
  ./bq_schema.json

Waiting on bqjob_r5e5b89642efeba7a_0000019547d4f119_1 ... (1s) Current status: DONE   
